In [1]:
!pip install transformers
!pip install miditok
!pip install tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 39.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 50.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.7/93.7 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.1/51.1 kB 5.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [1]:
import tensorflow as tf
import transformers
import miditok
import numpy
import os
import pathlib
from tensorflow import keras
import json
from transformers import TFGPT2LMHeadModel, GPT2Config, Trainer, TrainingArguments, GenerationConfig
from miditok import MIDILike, MIDITokenizer
from miditok.constants import CHORD_MAPS
from miditoolkit import MidiFile
import tqdm

In [2]:
attn_window = 10
j = 0
input_seq = [] 
target_seq = []
batch_size = 16
buffer_size = 1000


In [4]:
from google.colab import drive

drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [6]:
directory = 'C:/Users/simas/Downloads/dummy dataset/BPE'

for fname in os.listdir(directory):                     # for each file in the directory
    with open(os.path.join(directory, fname)) as i:     # open the file
        data = json.load(i)                             # load the json file                             

    for i in range(0, len(data['ids'][0])-1, 11):    # for each token in the file
            x = data['ids'][0][i:i+attn_window]      # get the next 10 tokens
            y = data['ids'][0][i+1:i+attn_window+1]  # get the next 1+10 tokens

            if len(x) == attn_window and len(y) == attn_window: # if the length of the tokens is 10
                
                print('batch number:', j)            # print the batch number
                print('input:', x)                   
                print('target:', y) 
                print('-----')

                j += 1                               # increment the batch number

                input_seq.append(x)                  # append the input sequence
                target_seq.append(y)                 # append the target sequence

batch number: 168
input: [65, 53, 6, 51, 9, 51, 13, 51, 52, 30]
target: [53, 6, 51, 9, 51, 13, 51, 52, 30, 33]
-----
batch number: 169
input: [37, 55, 8, 51, 11, 51, 15, 51, 52, 32]
target: [55, 8, 51, 11, 51, 15, 51, 52, 32, 35]
-----
batch number: 170
input: [39, 5, 51, 10, 51, 13, 51, 52, 29, 34]
target: [5, 51, 10, 51, 13, 51, 52, 29, 34, 37]
-----
batch number: 171
input: [3, 51, 5, 51, 12, 51, 52, 27, 29, 36]
target: [51, 5, 51, 12, 51, 52, 27, 29, 36, 53]
-----
batch number: 172
input: [6, 51, 9, 51, 13, 51, 52, 30, 33, 37]
target: [51, 9, 51, 13, 51, 52, 30, 33, 37, 55]
-----
batch number: 173
input: [8, 51, 11, 51, 15, 51, 52, 32, 35, 39]
target: [51, 11, 51, 15, 51, 52, 32, 35, 39, 5]
-----
batch number: 174
input: [51, 10, 51, 13, 51, 52, 29, 34, 37, 3]
target: [10, 51, 13, 51, 52, 29, 34, 37, 3, 51]
-----
batch number: 175
input: [65, 62, 15, 51, 18, 51, 22, 51, 52, 39]
target: [62, 15, 51, 18, 51, 22, 51, 52, 39, 42]
-----
batch number: 176
input: [46, 64, 17, 51, 20, 51, 

In [7]:
print('input_seq length:', len(input_seq))
print('target_seq length:', len(target_seq))

input_seq length: 252
target_seq length: 252


In [8]:
from sklearn.model_selection import train_test_split

train_size = 0.8 # 80/20 split between train and test+val data
test_size = 0.5 # 50/50 (80/10/10) split between train, test and val

x_train, x_rem, y_train, y_rem = train_test_split(input_seq, target_seq, train_size = 0.8)
x_test, x_val, y_test, y_val = train_test_split(x_rem, y_rem, test_size = 0.5)


In [9]:
print('train input:', len(x_train)), print('train target:', len(y_train))
print('val input:', len(x_val)), print('val target:', len(y_val))
print('test input:', len(x_test)), print('test target:', len(y_test))

train input: 201
train target: 201
val input: 26
val target: 26
test input: 25
test target: 25


(None, None)

In [10]:
pitch_range = range(21, 109)
beat_res = {(0, 4): 8, (4, 12): 4}
nb_velocities = 32
additional_tokens = {'Chord': True, 'Rest': True, 'Tempo': True,
                     'rest_range': (2, 8),  # (half, 8 beats)
                     'nb_tempos': 32,  # nb of tempo bins
                     'tempo_range': (40, 250),  # (min, max)
                     'Program': False,
                     "chord_maps": CHORD_MAPS,
                     "chord_tokens_with_root_note": True,
                     "chord_unknown": False}
special_tokens = ["PAD", "BOS", "EOS"]

In [11]:
#creates tokenizer
tokenizer = MIDILike(pitch_range, beat_res, nb_velocities, additional_tokens, special_tokens=special_tokens)

In [19]:
# Creates model
config = GPT2Config( 
    vocab_size=500,
    n_positions=attn_window,
    n_embd=64,
    n_layer=4,
    n_head=4,
    n_inner=128,
    resid_pdrop=.1,
    embd_pdrop=.1,
    attn_pdrop=.1,
    padding_token_id=tokenizer['PAD_None'],
    bos_token_id=tokenizer['BOS_None'],
    eos_token_id=tokenizer['EOS_None'],
)
model = TFGPT2LMHeadModel(config)

In [20]:
#model.from_pretrained("/content/gdrive/MyDrive/model_f/model2/")

In [21]:
# defining our optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0)
# definining our loss function
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
# defining our metric which we want to observe
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
# compiling the model
model.compile(optimizer=optimizer, loss=[loss, *[None] * model.config.n_layer], metrics=[metric])


In [23]:
num_epoch = 1000

# Create a callback that saves the model's weights
cp_callback = [

# tf.keras.callbacks.ModelCheckpoint("/content/gdrive/MyDrive/models_1/",
#                                                  verbose=1)

tf.keras.callbacks.LambdaCallback(on_epoch_end=lambda epoch, logs: model.save_pretrained("C:/Users/simas/Downloads/dummy dataset/models/model_dummy/"))
]

history = model.fit(x_train, y_train, epochs=num_epoch, callbacks=[cp_callback], validation_data = (x_val, y_val))


Epoch 1/1000
7/7 [==============================] - 1s 96ms/step - loss: 5.7738 - accuracy: 0.2323 - val_loss: 5.7312 - val_accuracy: 0.2577
Epoch 2/1000
7/7 [==============================] - 0s 45ms/step - loss: 5.7523 - accuracy: 0.2458 - val_loss: 5.7111 - val_accuracy: 0.2654
Epoch 3/1000
7/7 [==============================] - 0s 42ms/step - loss: 5.7316 - accuracy: 0.2517 - val_loss: 5.6906 - val_accuracy: 0.2731
Epoch 4/1000
7/7 [==============================] - 0s 41ms/step - loss: 5.7103 - accuracy: 0.2532 - val_loss: 5.6708 - val_accuracy: 0.2769
Epoch 5/1000
7/7 [==============================] - 0s 39ms/step - loss: 5.6882 - accuracy: 0.2682 - val_loss: 5.6507 - val_accuracy: 0.2769
Epoch 6/1000
7/7 [==============================] - 0s 45ms/step - loss: 5.6689 - accuracy: 0.2711 - val_loss: 5.6303 - val_accuracy: 0.2769
Epoch 7/1000
7/7 [==============================] - 0s 42ms/step - loss: 5.6485 - accuracy: 0.2677 - val_loss: 5.6106 - val_accuracy: 0.2769
Epoch 8/1000
